# Quantum Circuit Equivalence - Toffoli - U3 Gates Parameters
**Name:** Ana Elisa Franco Flores\
**Date:** October 22nd, 2025\
**Description:** This project demonstrates a fundamental concept in quantum computing: **circuit equivalence**. It finds the parameters for two universal U3 gates that, when combined with a specific arrangement of CNOT gates, perfectly simulate the behavior of a three-qubit Toffoli (CCX) gate.

Finding the 6 parameters (θ₁, φ₁, λ₁, θ₂, φ₂, λ₂) for the U3 gates to simulate a Toffoli one.

In [7]:
# ------------------------------------------------------------
# Required libraries
# ------------------------------------------------------------

!pip install numpy qiskit matplotlib scipy qiskit[visualization]

import numpy as np
import qiskit
from qiskit import QuantumCircuit, transpiler
from qiskit.visualization import circuit_drawer, plot_histogram
from qiskit.quantum_info import Operator
from qiskit.circuit import Parameter, parametervector
from scipy.optimize import minimize
from math import pi, cos, sin, exp
import matplotlib.pyplot as plt
import matplotlib as mpl 
import random

%matplotlib inline
plt.style.use('default')

print ("Libraries imported successfully.")

Libraries imported successfully.
Libraries imported successfully.


## Defining the Toffoli Gate 

We need to define the Toffoli (CCX) gate on 3 qubits as shown on the circuit below, obtaining its circuit representation and its matrix form (unitary operator).

In [8]:
# ------------------------------------------------------------
# Defining the Toffoli (CCX) gate and getting its matrix
# ------------------------------------------------------------
qc_toffoli = QuantumCircuit(3)
qc_toffoli.ccx(0, 1, 2)

# Matrix representation
U_toffoli = Operator(qc_toffoli).data

print("The Toffoli gate:")
print(qc_toffoli.draw(output='text'))

print("\nToffoli gate matrix:")
print(U_toffoli)

The Toffoli gate:
          
q_0: ──■──
       │  
q_1: ──■──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘

Toffoli gate matrix:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


## Build the Parameterized Circuit

Now, we construct a **parameterized quantum circuit** that uses two U3 gates with parameters θ, φ, λ, and a series of CNOT, T and T' gates.  

This circuit will later be optimized so that it behaves like the Toffoli gate.

In [9]:
# ------------------------------------------------------------
# Defining parameters for U3(1) and U3(2)
# ------------------------------------------------------------
theta1, phi1, lam1 = Parameter('θ₁'), Parameter('φ₁'), Parameter('λ₁')
theta2, phi2, lam2 = Parameter('θ₂'), Parameter('φ₂'), Parameter('λ₂')

qc_param = QuantumCircuit(3)

#------------------------------------------------------------
# Building circuit
# ------------------------------------------------------------

qc_param.t(0)
qc_param.u(theta1, phi1, lam1, 2)
qc_param.cx(0,1)
qc_param.tdg(1)
qc_param.cx(0,1)
qc_param.t(1)
qc_param.cx(1,2)
qc_param.tdg(2)
qc_param.cx(0,2)
qc_param.t(2)
qc_param.cx(1,2)
qc_param.u(theta2, phi2, lam2, 2)
qc_param.cx(0,2)
qc_param.t(2)
qc_param.h(2)

print(" Parameterized Circuit:")
print(qc_param.draw(output='text'))

 Parameterized Circuit:
          ┌───┐                                                      »
q_0: ─────┤ T ├───────■───────────■─────────────────────■────────────»
          └───┘     ┌─┴─┐┌─────┐┌─┴─┐┌───┐              │            »
q_1: ───────────────┤ X ├┤ Tdg ├┤ X ├┤ T ├──■───────────┼─────────■──»
     ┌─────────────┐└───┘└─────┘└───┘└───┘┌─┴─┐┌─────┐┌─┴─┐┌───┐┌─┴─┐»
q_2: ┤ U(θ₁,φ₁,λ₁) ├──────────────────────┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├»
     └─────────────┘                      └───┘└─────┘└───┘└───┘└───┘»
«                                   
«q_0: ─────────────────■────────────
«                      │            
«q_1: ─────────────────┼────────────
«     ┌─────────────┐┌─┴─┐┌───┐┌───┐
«q_2: ┤ U(θ₂,φ₂,λ₂) ├┤ X ├┤ T ├┤ H ├
«     └─────────────┘└───┘└───┘└───┘


## Defining the Cost Function

Function that measures how different our parameterized circuit is from the true Toffoli gate.  
The cost will be minimized by adjusting the parameters.

In [10]:
# ------------------------------------------------------------
# Cost function: difference between current circuit and Toffoli
# ------------------------------------------------------------
def cost(params):
    """Compute the distance between the parameterized circuit
    and the Toffoli gate (Frobenius norm)."""
    
    param_dict = {
        theta1: params[0], phi1: params[1], lam1: params[2],
        theta2: params[3], phi2: params[4], lam2: params[5]
    }

    qc_bound = qc_param.assign_parameters(param_dict)
    U_current = Operator(qc_bound).data

    diff = U_current - U_toffoli
    return np.linalg.norm(diff)

## Run the Optimization

We start with random initial values for all six parameters and use **Powell's optimization method** to minimize the cost function.

In [11]:
# ------------------------------------------------------------
# Optimization routine
# ------------------------------------------------------------
initial_params = np.random.rand(6) * 2 * np.pi

print(f"Initial parameters: {initial_params}")
print(f"Initial cost: {cost(initial_params)}")

# Running optimization
result = minimize(cost, initial_params, method='powell', tol=1e-6)

optimized_params = result.x
final_cost = result.fun

print("Optimization complete")
print(f"Optimized parameters: {optimized_params}")
print(f"Final cost: {final_cost}")

# Rounding parameters for readability
rounded_params = np.round(optimized_params / pi, 4) * pi
print(f"Rounded parameters (in multiples of π): {rounded_params/pi} * π")

Initial parameters: [0.19780121 5.66901918 0.61596871 4.76536184 0.78043257 6.00341868]
Initial cost: 4.27922039922046
Optimization complete
Optimized parameters: [-4.71238898  6.28318531  3.14159265  6.28318531 -0.43669634  5.93448349]
Final cost: 5.496545205106476e-12
Rounded parameters (in multiples of π): [-1.5    2.     1.     2.    -0.139  1.889] * π
Optimization complete
Optimized parameters: [-4.71238898  6.28318531  3.14159265  6.28318531 -0.43669634  5.93448349]
Final cost: 5.496545205106476e-12
Rounded parameters (in multiples of π): [-1.5    2.     1.     2.    -0.139  1.889] * π


In [12]:
# ------------------------------------------------------------
# Constructing and verify the final circuit
# ------------------------------------------------------------
final_param_dict = {
    theta1: optimized_params[0], phi1: optimized_params[1], lam1: optimized_params[2],
    theta2: optimized_params[3], phi2: optimized_params[4], lam2: optimized_params[5]
}

qc_final = qc_param.assign_parameters(final_param_dict)
U_final = Operator(qc_final).data

print("Final circuit matrix:")
print(np.round(U_final, 4))

print("Toffoli matrix:")
print(np.round(U_toffoli, 4))

# Verify equivalence
equal_matrices = np.allclose(U_final, U_toffoli, atol=1e-5)
print(f"Equivalent matrices? {equal_matrices}")

Final circuit matrix:
[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j -0.-0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j -0.-0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [ 0.-0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.-0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j]]
Toffoli matrix:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+

## Conclusion

Successfully optimized the parameters of a **U3-based circuit** to reproduce the **Toffoli gate**.

This demonstrates the concept of **circuit equivalence**: that a universal gate set (like U3, T, T' and CNOT) can represent any unitary operation.

**Key takeaways:**
- The optimization converges to parameters close to multiples of π.
- The final matrix matches the Toffoli gate.
- This approach can be extended to other multi-qubit gates.

**Final Conclusion**: This project successfully establishes the foundations of quantum gate tomography, demonstrating both theoretically and computationally the universality of specific quantum gate sets. 

**The precise parameter values obtained from optimization are:**

**First U3 Gate:**
- θ₁ = 1.5π 
- φ₁ = 2π  
- λ₁ = 1.0π 

**Second U3 Gate:**
- θ₂ = 2.0π 
- φ₂ = 0.139π 
- λ₂ = 1.89π 

These represent the exact mathematical solution for Toffoli gate decomposition using elementary quantum operations with a final cost of **5.496545e-12** (convergence achieved).